In [1]:
from genepy.utils import helper as h

from taigapy import TaigaClient
from celligner import Celligner
tc = TaigaClient()
from bokeh.plotting import output_notebook
from depmapomics import tracker as track
#autoreload
%load_ext autoreload
%autoreload 2
output_notebook()

/home/jeremie/celligner/celligner/mnnpy/mnnpy/utils.py:14: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "l2_norm" failed type inference due to: No implementation of function Function(<function norm at 0x7f92ac0738c8>) found for signature:
 
 >>> norm(x=array(float32, 2d, A), axis=Literal[int](1))
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function 'norm_impl': File: numba/np/linalg.py: Line 2352.
    With argument(s): '(x=array(float32, 2d, A), axis=int64)':
   Rejected as the implementation raised a specific error:
     TypeError: norm_impl() got an unexpected keyword argument 'x'
  raised from /home/jeremie/miniconda3/lib/python3.7/site-packages/numba/core/typing/templates.py:722

During: resolving callee type: Function(<function norm at 0x7f92ac0738c8>)
During: typing of call at /home/jeremie/celligner/celligner/mnnpy/mnnpy/utils.py (16)


File "celligner/mnnpy/mnnpy/utils.py", li

/home/jeremie/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Loading BokehJS ...

In [2]:
# load from taiga public (figshare link)
# load internal expression,
CCLE_expression = tc.get(name='internal-21q3-fe4c',
                         file='CCLE_expression_full')  # https://depmap.org/portal/download/
# load  TCGA expression
# this dataset was generated from  ,using this script: 
# caan be found here:
TCGA_expression = tc.get(name='celligner-input-9827',
                         file='tumor_expression')

No dataset version provided. Using version 16.
No dataset version provided. Using version 1.


In [3]:
# subset gene names to ensembl ids only
CCLE_expression = CCLE_expression[CCLE_expression.columns[:-92]]
CCLE_expression.columns = list(map(lambda x: x.split(
    ' (')[1][:-1] if ' (' in x else x, CCLE_expression.columns))

common = set(CCLE_expression.columns).intersection(
    set(TCGA_expression.columns))
CCLE_expression = CCLE_expression[list(common)]
TCGA_expression = TCGA_expression[list(common)]

In [4]:
# loading annotations
CCLE_annotation = track.getTracker() # the function uses pygsheets to load this: https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY/edit#gid=746987867 
TCGA_annotation = tc.get(name='celligner-input-9827',
                         file='tumor_annotations') # generated manually, found also here: 

No dataset version provided. Using version 1.


In [5]:
# transforming annotations
CCLE_annotation = CCLE_annotation.drop_duplicates('arxspan_id').set_index("arxspan_id")
CCLE_annotation = CCLE_annotation.loc[CCLE_expression.index, ["origin", 'subtype']].rename(columns={"origin": "tissue_type", "subtype": 'disease_type'})
CCLE_annotation["cell_type"] = "cancer cell line"

TCGA_annotation = TCGA_annotation.set_index("sampleID").loc[TCGA_expression.index,["lineage",
"subtype"]].rename(columns={"lineage":"tissue_type", "subtype": 'disease_type'})
TCGA_annotation['cell_type'] = "tumor sample"

In [6]:
my_alligner = Celligner(snn_kwargs={"neighbor_num": 30, "min_shared_neighbor_proportion": 1/15,}, make_plots=True)

my_alligner.fit(TCGA_expression, TCGA_annotation)

fetching gene names from biomart cache
using only usefull genes
looking at 12236 samples.
found 29593 common genes
creating a fit dataset..
reducing dimensionality...
clustering...
         Falling back to preprocessing with `sc.pp.pca` and default params.


doing differential expression analysis on the clusters
running differential expression on 33 clusters
running limmapy on the samples
you need to have R installed with the limma library installed
3.2.1
done


In [ ]:
my_alligner.transform(CCLE_expression, CCLE_annotation)

looking at 1411 samples.
found 29593 common genes
creating a transform input..
reducing dimensionality...


In [14]:
my_alligner.save('temp/demo/')

In [15]:
! ls -alh temp/demo

total 4.6G
drwxr-xr-x 2 jeremie jeremie 4.0K Sep 17 20:18 .
drwxr-xr-x 4 jeremie jeremie 4.0K Sep 17 20:18 ..
-rw-r--r-- 1 jeremie jeremie 4.6G Sep 17 20:19 model.pkl


In [ ]:
my_alligner.load('temp/demo/')

In [27]:
my_alligner.plot()